In [1]:
import mlrun
import os
mlrun.__version__

'1.6.1'

In [2]:
project_name = "nodegroup-test"
project = mlrun.get_or_create_project(project_name, context="./")
git_source =              "git://github.com/xsqian/mlrun-github-example.git#main"
project.set_source(source=git_source, pull_at_runtime=True)
project.save()

> 2024-06-07 16:50:06,451 [info] Loading project from path: {'project_name': 'nodegroup-test', 'path': './'}
> 2024-06-07 16:50:21,964 [info] Project loaded successfully: {'project_name': 'nodegroup-test', 'path': './', 'stored_in_db': True}


In [3]:
%%writefile functions/nodegroup_test.py

import time
import psutil


def nodegroup_test(context):
    context.logger.info("Resource Test!")
    mem = psutil.virtual_memory()
    vcpus = psutil.cpu_count(logical=True)
    pcpus = psutil.cpu_count(logical=False)
    context.logger.info(f"CPUs virtual  -> {vcpus}")
    context.logger.info(f"CPUs physical -> {pcpus}")
    context.logger.info(
        f"Memory avlbl  -> {round(mem.available/(1024 * 1024 * 1024), 2)}GB"
    )
    context.logger.info(f"Memory used   -> {round(mem.used/(1024 * 1024 * 1024), 2)}GB")
    context.logger.info(f"Sleeping 300s")


Overwriting functions/nodegroup_test.py


In [4]:
func2 = project.set_function(
    name="nodegroup-test",
    func="functions/nodegroup_test.py",
    kind="job",
    image="mlrun/mlrun:1.6.1",
    handler="nodegroup_test",
    with_repo=True,
)
func2.with_requests(mem="16G", cpu=2)
func2.with_limits(mem="32G", cpu=4)
func2.save()

'db://nodegroup-test/nodegroup-test:latest'

In [5]:
print(func2.to_yaml())

kind: job
metadata:
  name: nodegroup-test
  tag: latest
  project: nodegroup-test
  categories: []
spec:
  command: functions/nodegroup_test.py
  args: []
  image: mlrun/mlrun:1.6.1
  build:
    source: ./
    commands: []
    code_origin: git://github.com/xsqian/mlrun-github-example.git#refs/heads/main#68045eb18ed90f26e8ba469a4258780b59677a5b
    requirements: []
  description: ''
  default_handler: nodegroup_test
  disable_auto_mount: false
  clone_target_dir: ''
  volumes: []
  volume_mounts: []
  env: []
  resources:
    requests:
      memory: 16G
      cpu: 2
    limits:
      memory: 32G
      cpu: 4
  node_selector:
    app.iguazio.com/node-group: ondemand
  priority_class_name: igz-workload-medium
  preemption_mode: prevent
  state_thresholds:
    pending_scheduled: 1h
    pending_not_scheduled: '-1'
    image_pull_backoff: 1h
    executing: 24h
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpres

In [16]:
project.run_function(func2)

> 2024-06-07 17:59:18,469 [info] Storing function: {'name': 'nodegroup-test-nodegroup-test', 'uid': 'f273d98c4a804a969cd828e43e43bc10', 'db': 'http://mlrun-api:8080'}
> 2024-06-07 17:59:18,727 [info] Job is running in the background, pod: nodegroup-test-nodegroup-test-47s86
> 2024-06-07 17:59:21,738 [info] extracting source from git://github.com/xsqian/mlrun-github-example.git#main to /mlrun/code
> 2024-06-07 17:59:21,799 [info] Resource Test!
> 2024-06-07 17:59:21,800 [info] CPUs virtual  -> 16
> 2024-06-07 17:59:21,800 [info] CPUs physical -> 8
> 2024-06-07 17:59:21,800 [info] Memory avlbl  -> 47.44GB
> 2024-06-07 17:59:21,800 [info] Memory used   -> 11.57GB
> 2024-06-07 17:59:21,800 [info] Sleeping 300s
> 2024-06-07 17:59:21,847 [info] To track results use the CLI: {'info_cmd': 'mlrun get run f273d98c4a804a969cd828e43e43bc10 -p nodegroup-test', 'logs_cmd': 'mlrun logs f273d98c4a804a969cd828e43e43bc10 -p nodegroup-test'}
> 2024-06-07 17:59:21,847 [info] Or click for UI: {'ui_url': 'h

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
nodegroup-test,...3e43bc10,0,Jun 07 17:59:21,completed,nodegroup-test-nodegroup-test,v3io_user=xingshengkind=jobowner=xingshengmlrun/client_version=1.6.1mlrun/client_python_version=3.9.18host=nodegroup-test-nodegroup-test-47s86,,,,


> 2024-06-07 17:59:29,880 [info] Run execution finished: {'status': 'completed', 'name': 'nodegroup-test-nodegroup-test'}


In [58]:
%%writefile workflow/workflow.py
""" The main workflow of the project

This file creates the MLRun pipeline to run the functions
in an orchestrated manner

"""

from kfp import dsl
import mlrun

# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(name="testing-resource")
def test():
    project = mlrun.get_current_project()
    func = project.get_function("nodegroup-test")
    run2 = mlrun.run_function(
        name="nodegroup-test-run-from-workflow",
        function=func,
        auto_build=False,
        local=False,
    )

Overwriting workflow/workflow.py


In [59]:
project.set_workflow(
    name="workflow", workflow_path="workflow/workflow.py", handler="test"
)
project.save()

In [61]:
project.run(
    name="workflow",
    schedule="*/10 * * * *",
    dirty=True,
)

> 2024-06-07 22:02:08,131 [warning] WARNING!, You seem to have uncommitted git changes, use .push()
> 2024-06-07 22:02:08,133 [info] Submitting remote workflow: {'workflow_engine': 'kfp', 'schedule': '*/10 * * * *', 'project_name': 'nodegroup-test'}
> 2024-06-07 22:02:08,343 [info] Workflow scheduled successfully: {'workflow_response': WorkflowResponse(project='nodegroup-test', name='workflow', status='scheduled', run_id=None, schedule='*/10 * * * *')}
